<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/Optimization_Project3_Recomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
#Import table
df_inovmovie = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/condition_gi.pickle')
df_inovmovie

,primaryName,tconst,genres,primaryTitle,averageRating,numVotes,primaryProfession,titleType,startYear,wheighted_IMDB
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,cinematographer,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Short,Carmencita,5.7,1922,cinematographer,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,director,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Short,Carmencita,5.7,1922,director,short,1894,6.278611
1,William K.L. Dickson,tt0000001,Documentary,Carmencita,5.7,1922,producer,short,1894,6.278611
...,...,...,...,...,...,...,...,...,...,...
3444278,Oliver Bickers,tt9916544,Drama,My Sweet Prince,6.9,60,actor,short,2019,6.324476
3444278,Oliver Bickers,tt9916544,Short,My Sweet Prince,6.9,60,actor,short,2019,6.324476
3444279,Ganesh Vasant Patil,tt9916730,\N,6 Gunn,9.2,7,actor,movie,2017,6.323900
3444281,Ujjwala Gawde,tt9916730,\N,6 Gunn,9.2,7,producer,movie,2017,6.323900


In [ ]:
#NList of columns to choose from for the model
df_inovmovie.columns

Index(['primaryTitle', 'primaryName', 'tconst', 'genres', 'primaryProfession',
       'titleType', 'startYear', 'wheighted_IMDB', 'Action', 'Adult',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War',
       'Western', '\N'],
      dtype='object')

In [ ]:
#KNN model

X = df_inovmovie[['startYear', 'wheighted_IMDB',
       'Adventure']]

model = NearestNeighbors(n_neighbors=5).fit(X)

In [ ]:
name = ['The Shawshank Redemption']

array1, array2 = model.kneighbors(df_inovmovie.loc[df_inovmovie['primaryTitle'].isin(name), ['startYear', 'wheighted_IMDB', 'Adventure']])
array1, array2

(array([[0.        , 0.40354234, 0.50196728, 0.79117119, 0.818433  ]]),
 array([[ 0,  7, 10, 29, 34]]))

In [ ]:
list_1 = array1.tolist()
list_2 = array2.tolist()

flat_list1 = list(np.concatenate(list_1).flat)
flat_list2 = list(np.concatenate(list_2).flat)

d = {'Distance': flat_list1,'index': flat_list2}
df12 = pd.DataFrame(d)
df12

dfnl_df12 = pd.merge(df_inovmovie, df12, how='inner', on=["index", "index"])
dfnl_df12.sort_values(by = 'Distance').head(5)

,index,primaryTitle,primaryName,tconst,genres,primaryProfession,titleType,startYear,wheighted_IMDB,Action,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N,Distance
0,0,The Shawshank Redemption,William Sadler,tt0111161,Drama,producer,movie,1994,9.272273,0,...,0,0,0,0,0,0,0,0,0,0.000000
1,7,Pulp Fiction,Uma Thurman,tt0110912,Drama,producer,movie,1994,8.868730,0,...,0,0,0,0,0,0,0,0,0,0.403542
2,10,Forrest Gump,Winston Groom,tt0109830,Romance,producer,movie,1994,8.770305,0,...,1,0,0,0,0,0,0,0,0,0.501967
3,29,It's a Wonderful Life,Tony Ka Fai Leung,tt0109524,Fantasy,producer,movie,1994,8.481102,0,...,0,0,0,0,0,0,0,0,0,0.791171
4,34,Léon: The Professional,Thierry Arbogast,tt0110413,Drama,producer,movie,1994,8.453840,1,...,0,0,0,0,0,0,0,0,0,0.818433


In [ ]:
#Remove the movie that is to be recomended from the list
name = ('The Shawshank Redemption')

condition_gi_nn = df_inovmovie[df_inovmovie['primaryTitle'].str.contains(name) == False]
# condition_gi_nn = condition_gi.loc[condition_gi['primaryTitle'] != name]

name1 = ['The Shawshank Redemption']

X1 = condition_gi_nn[['startYear', 'wheighted_IMDB',
       'Adventure']]

model = NearestNeighbors(n_neighbors=5).fit(X1)

array1, array2 = model.kneighbors(df_inovmovie.loc[df_inovmovie['primaryTitle'].isin(name1), ['startYear', 'wheighted_IMDB', 'Adventure']])
array1, array2

list_1 = array1.tolist()
list_2 = array2.tolist()

flat_list1 = list(np.concatenate(list_1).flat)
flat_list2 = list(np.concatenate(list_2).flat)

d = {'Distance': flat_list1,'index': flat_list2}
df12 = pd.DataFrame(d)
df12

dfnl_df12 = pd.merge(condition_gi_nn, df12, how='inner', on=["index", "index"])
dfnl_df12.sort_values(by = 'Distance').head(5)

,index,primaryTitle,primaryName,tconst,genres,primaryProfession,titleType,startYear,wheighted_IMDB,Action,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N,Distance
1,6,12 Angry Men,Sidney Lumet,tt0050083,Drama,producer,movie,1957,8.917393,0,...,0,0,0,0,0,0,0,0,0,0.403542
2,9,Fight Club,Zach Grenier,tt0137523,Drama,producer,movie,1999,8.770918,0,...,0,0,0,0,0,0,0,0,0,0.501967
3,28,Life Is Beautiful,Zita Kabátová,tt3881136,Romance,producer,short,2022,8.520547,0,...,1,0,0,0,0,0,0,0,0,0.791171
4,33,Back to the Future,Robert Zemeckis,tt0088763,Sci-Fi,producer,movie,1985,8.455415,0,...,0,1,0,0,0,0,0,0,0,0.818433
0,3,The Lord of the Rings: The Return of the King,Viggo Mortensen,tt0167260,Drama,producer,movie,2003,8.963991,1,...,0,0,0,0,0,0,0,0,0,1.050274
